## Required Imports

In [1]:
# Used to track processing time
import time

# Use to write a file with the current datetime
import datetime
import os

from textwrap import dedent
from langchain_community.llms import Ollama
from crewai import Crew, Agent, Task

## Initialize LLMs

In [2]:
llm = Ollama(model="NEW_MODEL")

## Create Agents

In [3]:
class CrewAgents:

    def variable_name_agent(self):
        return Agent(
            role='Senior Python Developer',
            goal=dedent(
                '''
                To review Python functions and ensure that variable names are
                following best practices. Variable names are concise, descriptive,
                and easy to understand.
                '''
            ),
            backstory=dedent(
                '''
                You're a Python developer who's been programming for decades. You're
                renowned for writing excellent code, and particularly for writing
                high-quality variable names.
                '''),
            allow_delegation=False,
            verbose=True,
            llm=llm
    )

    def docstring_agent(self):
        return Agent(
            role='Senior Code Documentation Expert',
            goal='Write docstrings for Python functions.',
            backstory=dedent(
                '''
                You're a renowned code documentation expert, specifically in Python.
                You know how to write clean, concise docstrings that include all the
                information relevant to developers trying to understand a function.
                '''),
            allow_delegation=False,
            verbose=True,
            llm=llm
    )
    
    def quality_assurance_agent(self):
        return Agent(
            role="Senior Python Quality Assurance Tester",
            goal=dedent(
            '''
            To review Python code and ensure that it follows best practices.
            Double-check that variable names are accurate and concise and that
            the docstrings accurately reflect what the function actually does.
            '''
            ),
            backstory=dedent(
            '''
            You're a renowned quality assurance expert. You've been reviewing code
            for decades and ensure that code is flawless, with excellent variable
            names and docstrings.
            '''
            ),
            allow_delegation=False,
            verbose=True,
            llm=llm
        )
    
    def code_writer_agent(self):
        return Agent(
            role="Senior Code Writer",
            goal=dedent(
            '''
            Take code and strip out everything except the code
            and docstrings.
            '''
            ),
            backstory=dedent(
            '''
            You do an amazing job of getting code and stripping it
            of everything that's not code or documentation.
            '''
            ),
            allow_delegation=False,
            verbose=True,
            llm=llm
        )
        

## Create Tasks

In [4]:
class CrewTasks:

    def improve_variable_names(self, agent, provided_code):
        return Task(
            description=dedent(
                f'''
                Review the variable names in the Python code. Update
                the variable names so that they're accurate, concise, and
                easy to read and understand. Update the variable names only;
                don't change anything else about the functions, and do not
                add comments anywhere in the code.
                
                Python Function
                ---------------
                {provided_code}    
                '''
            ),
            expected_output=dedent(
                '''
                A Python function with variable names that are 
                accurate, concise, and easy to understand.
                '''
            ),
            agent=agent
        )
    
    def add_docstring(self, agent, provided_code):
        return Task(
            description=dedent(
                f'''
                Provide docstrings to the Python code provided below.
                Ensure the documentation is concise, accurate, and easy
                to read.
                
                Python Function
                ---------------
                {provided_code}    
                '''
            ),
            expected_output=dedent(
                '''
                A Python function with a well-written, concise, and
                easy to understand docstring.
                '''
            ),
            agent=agent
        )
    
    def review_code(self, agent, provided_code):
        return Task(
            description=dedent(
                f'''
                Review code and documentation and ensure it meets the 
                highest standards of quality. The variable names are
                clear, concise, and accurate; the docstrings are 
                accurate and descriptive, so that any developer reading
                them, even a junior developer, would understand exactly
                what they do and how to use them.
                
                Python Function
                ---------------
                {provided_code}    
                '''
            ),
            expected_output=dedent(
                '''
                A Python function with excellent variable names and 
                well-written, concise, and easy to understand docstring.
                '''
            ),
            agent=agent
        )
    
    def write_code(self, agent, provided_code):
        return Task(
            description=dedent(
                f'''
                Review the provided code and remove anything that
                is not code or docstrings. This information will
                then be written to a file that can be immediately used
                within a Python script.
                
                Python Function
                ---------------
                {provided_code}    
                '''
            ),
            expected_output=dedent(
                '''
                Code that consists ONLY of functions and their
                docstrings. All other information is stripped out.
                '''
            ),
            agent=agent
        )
        

## Read File
Reads a file of functions, each immediately proceeded by a comment.

These functions are read line by line and added to the `functions` list
for further processing.

In [5]:
# Define relative file path
test_suite_file_path = 'test_suite/test_functions_raw.py'

# Save all captured functions to a list
functions = []

with open(test_suite_file_path, 'r', encoding='utf-8') as file:

    function_code      = ''
    capturing_function = False
    
    for line in file:
        
        # Check if the line is blank
        if line.strip() == '':
            
            # If the line is blank and we're capturing
            # a function, we've reached the end of the
            # current function; reset
            if capturing_function:
                functions.append(function_code.strip())
                function_code = ''
                capturing_function = False
        
        # If line isn't blank, start capturing function
        else:
            function_code += line
            capturing_function = True
    
    # If the file doesn't end in a blank line,
    # ensure the last function is saved.
    if capturing_function:
        functions.append(function_code.strip())

## Process Functions

In [ ]:
# Initialize Agents & Tasks
agents = CrewAgents()
tasks  = CrewTasks()

# Create Agents
variable_name_agent = agents.variable_name_agent()
docstring_agent     = agents.docstring_agent()
qa_agent            = agents.quality_assurance_agent()
code_writer_agent   = agents.code_writer_agent()

# Process Functions
updated_functions = []

for function in functions:
    
    # Create tasks
    improve_variable_names = tasks.improve_variable_names(variable_name_agent, function)
    add_docstrings         = tasks.add_docstring(docstring_agent, improve_variable_names)
    review_code            = tasks.review_code(qa_agent, add_docstrings)
    write_code_to_file     = tasks.write_code(code_writer_agent, review_code)

    # Create crew & assign tasks
    crew = Crew(
        agents = [
            variable_name_agent,
            docstring_agent,
            qa_agent,
            code_writer_agent
        ],
        tasks = [
            improve_variable_names,
            add_docstrings,
            review_code,
            write_code_to_file
        ],
        verbose=True
    )

    # Start code-improvement process
    start_time = time.perf_counter()
    output = crew.kickoff()
    end_time = time.perf_counter()
    
    updated_functions.append(output.strip())

    print("################################")
    print("## Refactored Function Output ##")
    print("################################")
    print(f">>> Time taken: {end_time - start_time:.2f} seconds <<<\n")


 [DEBUG]: == Working Agent: Senior Python Developer
 [INFO]: == Starting Task: 
                Review the variable names in the Python code. Update
                the variable names so that they're accurate, concise, and
                easy to read and understand. Update the variable names only;
                don't change anything else about the functions, and do not
                add comments anywhere in the code.

                Python Function
                ---------------
                # Calculate Fibonacci sequence
def f(n):
    a, b = 0, 1
    for _ in range(n):
        a, b = b, a + b
    return a    



> Entering new CrewAgentExecutor chain...
Thought: I'm excited to dive into this task and make sure those variable names are top-notch!

Final Answer:
```
def calculate_fibonacci_sequence(n):
    initial_value, next_value = 0, 1
    for _ in range(n):
        initial_value, next_value = next_value, initial_value + next_value
    return initial_value
```

> Finished c

2024-05-16 15:13:16,239 - 8450341568 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


I now can give a great answer!

Final Answer:
```
def calculate_fibonacci_sequence(n):
    """
    Calculate the Fibonacci sequence up to the nth term.

    Args:
        n (int): The number of terms in the Fibonacci sequence.

    Returns:
        int: The nth term in the Fibonacci sequence.

    Example:
        >>> calculate_fibonacci_sequence(5)
        3

    Description:
        This function calculates the Fibonacci sequence, a series of numbers where each subsequent term is the sum of the two preceding terms. It starts with 0 and 1, and returns the nth term in the sequence.
    """
    initial_value, next_value = 0, 1
    for _ in range(n):
        initial_value, next_value = next_value, initial_value + next_value
    return initial_value
```

> Finished chain.
 [DEBUG]: == [Senior Code Writer] Task output: ```
def calculate_fibonacci_sequence(n):
    """
    Calculate the Fibonacci sequence up to the nth term.

    Args:
        n (int): The number of terms in the Fibonacci se

I now can give a great answer!

Final Answer:
```
def merge_sorted_lists(left_list: list, right_list: list) -> list:
    """
    Merge two sorted lists into one sorted list.

    This function takes two sorted lists as input and returns a new sorted list that contains all elements from both input lists. The function assumes that the input lists are already sorted in ascending order.

    Args:
        left_list (list): The first sorted list.
        right_list (list): The second sorted list.

    Returns:
        list: A new sorted list containing all elements from both input lists.

    Example:
        >>> left_list = [1, 3, 5]
        >>> right_list = [2, 4, 6]
        >>> merged_list = merge_sorted_lists(left_list, right_list)
        >>> print(merged_list)  # Output: [1, 2, 3, 4, 5, 6]

    :param left_list: The first sorted list.
    :type left_list: list
    :param right_list: The second sorted list.
    :type right_list: list
    :return: A new sorted list containing all elemen

2024-05-16 15:16:18,311 - 8450341568 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


...

I now can give a great answer!

Final Answer:
```
def send_email(server: smtplib.SMTP, from_addr: str, to_addr: str, subject: str, body: str) -> None:
    """Sends an email using SMTP.

    Args:
        server (smtplib.SMTP): The SMTP server to use for sending the email.
        from_addr (str): The address of the sender.
        to_addr (str): The address of the recipient.
        subject (str): The subject line of the email.
        body (str): The content of the email.

    Returns:
        None

    Raises:
        Exception: If an error occurs while sending the email, a descriptive error message is printed to the console.
    """
    try:
        server.sendmail(from_addr, to_addr, subject, body)
    except Exception as e:
        print(f"Error sending email: {str(e)}")
```

> Finished chain.
 [DEBUG]: == [Senior Code Writer] Task output: ```
def send_email(server: smtplib.SMTP, from_addr: str, to_addr: str, subject: str, body: str) -> None:
    """Sends an email using SMTP.

   

I now can give a great answer!

Final Answer:
```
def send_email(username: str, password: str, to_address: str, message: str) -> None:
    """Send an email using the provided username, password, recipient address, and message.

    Args:
        username (str): The username used for authentication.
        password (str): The password used for authentication.
        to_address (str): The recipient's email address.
        message (str): The content of the email.

    Returns:
        None
    """
    used_tools = 0
    tools_errors = 0
    delegations = 0
    i18n = I18N(prompt_file=None)
    thread = None
    prompt_context = None
    description = "Provide docstrings to the Python code provided below.\n" \
                 "Ensure the documentation is concise, accurate, and easy\n" \
                 "to read.\n"

    def e(u: str, p: str, t: str, m: str) -> None:
        s = smtplib.SMTP('smtp.example.com', 587)
        s.starttls()
        s.login(u, p)
        s.sendmail(u, t, m)

2024-05-16 15:40:33,723 - 8450341568 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


I understand the importance of this task. As a Senior Code Writer, my goal is to strip out everything except code and docstrings from the provided Python function. Here's my thought process:

1. Review the provided code and identify the functions with their respective docstrings.
2. Remove all unnecessary information, such as variable declarations, comments, and other metadata.
3. Keep only the function definitions and their corresponding docstrings.

With that in mind, here's my Final Answer:

```
def send_email(username: str, password: str, to_address: str, message: str) -> None:
    """Send an email using the provided username, password, recipient address, and message.

    Args:
        username (str): The username used for authentication.
        password (str): The password used for authentication.
        to_address (str): The recipient's email address.
        message (str): The content of the email.

    Returns:
        None
    """

def e(u: str, p: str, t: str, m: str) -> N

I now can give a great answer!

Final Answer:

```
def fetch_json_data() -> dict:
    """
    Fetch JSON data from a web API.

    This function sends a GET request to the specified API URL,
    parses the response, and returns the processed data.

    Args:
        None

    Returns:
        A dictionary containing the parsed JSON data.
    """
    api_url = "https://api.example.com/data"
    response = requests.get(api_url)
    data = response.json()
    parsed_data = parse_json_response(data)
    return parsed_data

def parse_json_response(json_data: dict) -> dict:
    """
    Parse the JSON response from the API.

    This function extracts relevant information from the JSON data,
    processes it for analysis, and returns the result.

    Args:
        json_data (dict): The JSON data to be processed.

    Returns:
        A dictionary containing the processed data.
    """
    extracted_data = extract_relevant_info(json_data)
    processed_data = process_data_for_analysis(extracte

## Write Code to Output File

In [ ]:
# Check if directory exists; if not, create it
# This directory is relative to the location of this Jupyter Notebook
save_location_file_path = 'test_outputs'
if not os.path.exists(save_location_file_path):
    os.makedirs(save_location_file_path)

# Create custom datetime output, used for filename
current_datetime = datetime.datetime.now().strftime('%y-%m-%d_%H:%M:%S') + '.txt' 
output_filename  = 'function_output_' + current_datetime
full_file_path   = os.path.join(save_location_file_path, output_filename)

with open(full_file_path, 'w', encoding='utf-8') as f:
    for func in updated_functions:
        
        # Remove backticks and any remaining whitespaces from function
        cleaned_func = func.strip('`').strip()
        f.write(cleaned_func + "\n\n")
            
print(f'Functions successfully written to file "{full_file_path}".')